<a href="https://colab.research.google.com/github/iharashuuji/IRL_algrithm/blob/main/igo_learn_battle_fitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git remote add origin main https://github.com/iharashuuji/IRL_algrithm.git

fatal: not a git repository (or any of the parent directories): .git


In [ ]:
# ==============================================================================
# セクション0: 必要なライブラリのインポート
# ==============================================================================
# これらはGoogle Colabにプリインストールされているため、特別な準備は不要です。
import numpy as np
from scipy.optimize import linear_sum_assignment
from itertools import combinations
import pandas as pd

# ==============================================================================
# セクション1: データ定義（お手本と選手リスト）
# ==============================================================================

def define_players():
    """
    今回の学習で使用する、仮想の参加者リストを定義します。
    - rank: 数値が小さいほど強い（7段=1, 6段=2, ..., 15級=23）
    - priority_group: クライアントの指示に基づく優先度。数値が小さいほど優先度が高い。
    """
    players = [
        # priority_group: 1 (最優先強化選手)
        {'id': 1, 'name': '有田', 'rank': 12, 'affiliation': '阪公大', 'priority_group': 1, 'is_focus': True},
        # priority_group: 2 (強化選手)
        {'id': 2, 'name': '山田', 'rank': 15, 'affiliation': '阪公大', 'priority_group': 2, 'is_focus': True},
        {'id': 3, 'name': '宇治', 'rank': 16, 'affiliation': '阪公大', 'priority_group': 2, 'is_focus': True},
        # priority_group: 3 (自校レギュラー)
        {'id': 4, 'name': '礒島', 'rank': 3,  'affiliation': '阪公大', 'priority_group': 3, 'is_focus': False},
        {'id': 5, 'name': '吉岡', 'rank': 4,  'affiliation': '阪公大', 'priority_group': 3, 'is_focus': False},
        # priority_group: 4 (他校レギュラー)
        {'id': 6, 'name': '羽瀬', 'rank': 12, 'affiliation': '阪', 'priority_group': 4, 'is_focus': False},
        {'id': 7, 'name': '近藤', 'rank': 14, 'affiliation': '奈', 'priority_group': 4, 'is_focus': False},
        # priority_group: 5 (レギュラー資格者)
        {'id': 8, 'name': '玉木', 'rank': 1,  'affiliation': '立', 'priority_group': 5, 'is_focus': False},
        {'id': 9, 'name': '北山', 'rank': 11, 'affiliation': '京', 'priority_group': 5, 'is_focus': False},
        # priority_group: 6 (OB/OG/その他)
        {'id': 10, 'name': '真田', 'rank': 1,  'affiliation': '阪公大OB', 'priority_group': 6, 'is_focus': False},
        {'id': 11, 'name': '古家', 'rank': 2,  'affiliation': '阪公大OG', 'priority_group': 6, 'is_focus': False},
        {'id': 12, 'name': '大林', 'rank': 15, 'affiliation': '阪公大', 'priority_group': 6, 'is_focus': False},
        {'id': 13, 'name': '稲留', 'rank': 5,  'affiliation': '阪公大', 'priority_group': 6, 'is_focus': False},
        {'id': 14, 'name': '羽田', 'rank': 16, 'affiliation': '阪公大', 'priority_group': 2, 'is_focus': True}, # 強化選手
        {'id': 15, 'name': '堤', 'rank': 18, 'affiliation': '阪公大', 'priority_group': 6, 'is_focus': False},
        {'id': 16, 'name': '新川', 'rank': 30, 'affiliation': '阪公大', 'priority_group': 6, 'is_focus': False},
    ]
    return players

In [ ]:
def get_player_id_map():
    """選手名からIDを引くための辞書を返す"""
    # define_players()関数で定義した選手リストを使用
    players = define_players()
    player_id_map = {p['name']: p['id'] for p in players}

    # 画像に登場するが、選手リストにいない可能性のある名前も追加
    # (名前の揺れなどに対応するため)
    additional_names = {
        "堤": 15, "古家": 11, "礒島": 4, "北山": 9, "羽瀬": 6, "稲留": 13,
        "近藤": 7, "大林": 12, "玉木": 8, "真田": 10, "新川": 16, "山田": 2,
        "羽田": 14, "宇治": 3, "吉岡": 5, "有田": 1, "堤": 15, "山下": -1, # 山下さんはリストにいないので仮ID
        "堤": 15, "堤": 15, "真田": 10, "大森": -2, "小島": -3, "堀": -4,
        "近藤": 7, "羽田": 14, "真田": 10, "大林": 12, "新川": 16, "山田": 2,
        "羽瀬": 6, "稲留": 13, "古家": 11, "玉木": 8, "北山": 9, "吉岡": 5,
        "有田": 1, "礒島": 4, "宇治": 3, "指導": -5, "休み": -6, "x": -7
    }
    player_id_map.update(additional_names)
    return player_id_map
def define_expert_demonstration(pattern_select=1):
    """
    専門家が作成した「理想の対戦表（お手本）」を定義します。
    画像から読み取った2つのパターンを収録。

    引数:
        pattern_select (int): 1なら左のブロック、2なら右のブロックのお手本を使用
    """

    # 選手名からIDへの変換マップを取得
    PLAYER_ID_MAP = get_player_id_map()

    # --- パターン1（画像の左側ブロック） ---
    pattern1_by_name = {
        "玉木": ["有田", "礒島", "北山", "羽瀬", "古家"],
        "真田": ["羽瀬", "古家", "近藤", "吉岡", "山田"],
        "古家": ["吉岡", "羽瀬", "玉木", "真田", "北山"],
        "礒島": ["北山", "有田", "稲留", "古家", "玉木"],
        "吉岡": ["古家", "羽瀬", "玉木", "真田", "北山"],
        "有田": ["玉木", "礒島", "羽瀬", "羽田・山田", "礒島"],
        "羽瀬": ["真田", "古家", "有田", "玉木", "近藤"],
        "北山": ["礒島", "玉木", "有田", "稲留", "真田"],
        "稲留": ["礒島", "玉木", "礒島", "北山", "羽瀬"],
        "近藤": ["山田", "新川", "真田", "大林", "羽瀬"],
        "山田": ["近藤", "宇治", "新川", "玉木２", "真田"],
        "大林": ["堤", "古家", "山田", "近藤", "新川"],
        "宇治": ["羽田", "山田", "新川", "玉木２", "近藤"],
        "羽田": ["宇治", "x", "x", "x", "x"],
        "堤": ["大林", "近藤", "羽田", "休み", "大林"],
        "新川": ["x", "近藤", "山田", "宇治", "近藤"]
    }

    # --- パターン2（画像の右側ブロック） ---
    pattern2_by_name = {
        "玉木": ["有田", "礒島", "稲留", "羽瀬・大林", "古家"],
        "真田": ["羽瀬", "古家", "近藤", "吉岡", "山田"],
        "古家": ["吉岡", "羽瀬", "玉木", "真田", "玉木"],
        "礒島": ["北山", "玉木", "有田", "古家", "有田"],
        "吉岡": ["古家", "羽瀬", "北山", "真田", "羽瀬"],
        "有田": ["玉木", "玉木", "礒島", "古家", "礒島"],
        "羽瀬": ["真田", "古家", "稲留", "玉木", "吉岡"],
        "北山": ["礒島", "北山", "吉岡", "礒島", "真田"],
        "稲留": ["礒島", "x", "玉木", "北山", "羽瀬"],
        "近藤": ["山田", "古家２", "真田", "大林", "新川"],
        "山田": ["近藤", "近藤", "真田", "羽田", "山田"],
        "大林": ["堤", "古家２", "山田", "新川", "古家"],
        "宇治": ["羽田", "新川", "山田", "玉木２", "吉岡"],
        "羽田": ["宇治", "x", "x", "x", "x"],
        "堤": ["大林", "羽田", "大林", "休み", "近藤"],
        "新川": ["x", "宇治", "大林", "山田", "近藤"]
    }

    # 使用するパターンを選択
    target_pattern = pattern1_by_name if pattern_select == 1 else pattern2_by_name

    # 5局分の全ペアをIDのリストに変換
    all_expert_pairs = []
    player_names = list(target_pattern.keys())

    for round_idx in range(5): # 5局分ループ
        round_pairs = set()
        for player_name in player_names:
            # 相手の名前に「・」や「２」が含まれる場合は無視する
            opponent_name = target_pattern[player_name][round_idx]
            if "・" in opponent_name or "２" in opponent_name:
                continue

            # 選手IDを取得（見つからない、または対戦ではない項目は無視）
            p1_id = PLAYER_ID_MAP.get(player_name, -99)
            p2_id = PLAYER_ID_MAP.get(opponent_name, -99)

            if p1_id >= 0 and p2_id >= 0:
                # 重複を避けるため、IDをソートしてタプルにし、セットに追加
                round_pairs.add(tuple(sorted((p1_id, p2_id))))

        all_expert_pairs.append(list(round_pairs))

    # 今回は話を簡単にするため、1局目のお手本だけを返す
    # 複数局を学習する場合は、この関数の戻り値や学習ループの修正が必要
    return all_expert_pairs[0]

In [ ]:
# ==============================================================================
# セクション2: 特徴量エンジニアリング
# ==============================================================================

def get_feature_vector(p1, p2, past_matchups):
    """
    2人の選手情報を受け取り、そのペアの「良さ」を評価するための特徴量ベクトルを計算します。
    """
    # === ★★★ 修正箇所 ★★★ ===
    # 再戦チェックを「選手ID」から「選手名」に変更
    if tuple(sorted((p1['name'], p2['name']))) in past_matchups:
        return np.array([-1e9, 0, 0, 0, 0, 0, 0])

    # --- ソフト制約: 各特徴量を計算 ---

    # 特徴1: 内部対戦ペナルティ (同じ大学 or OB/OG同士)
    is_internal = p1['affiliation'] in ['阪公大', '阪公大OB', '阪公大OG'] and \
                  p2['affiliation'] in ['阪公大', '阪公大OB', '阪公大OG']
    f_internal_match = -1.0 if is_internal else 0.0

    # 特徴2: 棋力差ペナルティ
    f_rank_difference = -abs(p1['rank'] - p2['rank'])

    # 特徴3: 強化選手ボーナス (強化選手が格上と当たるか)
    is_focus_vs_stronger = (p1['is_focus'] and p1['rank'] > p2['rank']) or \
                           (p2['is_focus'] and p2['rank'] > p1['rank'])
    f_focus_vs_stronger = 1.0 if is_focus_vs_stronger else 0.0

    # 特徴4: 外部同士の対戦ペナルティ
    is_external_vs_external = p1['affiliation'] not in ['阪公大', '阪公大OB', '阪公大OG'] and \
                              p2['affiliation'] not in ['阪公大', '阪公大OB', '阪公大OG']
    f_external_vs_external = -1.0 if is_external_vs_external else 0.0

    # 特徴5: 優先度ボーナス (優先度が高い選手同士の対戦を評価)
    f_priority_bonus = ( (7 - p1['priority_group']) + (7 - p2['priority_group']) ) / 2.0
    # === ★★★ 追加する特徴量 ★★★ ===
    # 特徴6: 一方的な試合への超特大ペナルティ
    f_lopsided_match = -1.0 if abs(p1['rank'] - p2['rank']) >= 10 else 0.0

    # 特徴量7:  強化選手同士の内部対戦ペナルティ
    is_focus_internal_match = p1['is_focus'] and p2['is_focus'] and \
                              p1['affiliation'] == '阪公大' and \
                              p2['affiliation'] == '阪公大'
    f_focus_internal_match = -1.0 if is_focus_internal_match else 0.0



    return np.array([
        f_internal_match,
        f_rank_difference,
        f_focus_vs_stronger,
        f_external_vs_external,
        f_priority_bonus,
        f_lopsided_match, #  <-- 追加
        f_focus_internal_match
    ])

In [ ]:
# ==============================================================================
# セクション3: 最適化ソルバー（推論エンジン）- 修正版
# ==============================================================================

def solve_matching(players, weights, past_matchups):
    """
    現在の「重み(weights)」に基づいて、最適なマッチングを見つけ出します。
    【修正点】戻り値を選手のIDペアから、選手の情報(辞書)のペアに変更しました。
    """
    # この関数に渡された players リストを直接変更しないようにコピーを作成
    current_players = players.copy()

    # --- 参加者が奇数の場合、ダミープレイヤーを追加 ---
    if len(current_players) % 2 != 0:
        dummy_player = {
            'id': -1, 'name': '休み', 'university': 'DUMMY', 'rank': 999,
            'is_focus': False, 'priority_group': 99
        }
        current_players.append(dummy_player)

    num_players = len(current_players)

    # スコア行列を作成します。
    cost_matrix = np.full((num_players, num_players), 1e9)

    for i, j in combinations(range(num_players), 2):
        p1 = current_players[i]
        p2 = current_players[j]

        # ダミープレイヤーとの対戦スコアは0（ニュートラル）にする
        if p1['id'] == -1 or p2['id'] == -1:
            score = 0
        else:
            features = get_feature_vector(p1, p2, past_matchups)
            score = np.dot(weights, features)

        cost_matrix[i, j] = -score
        cost_matrix[j, i] = -score

    # 最適化ソルバーを実行
    row_ind, col_ind = linear_sum_assignment(cost_matrix)

    # === ★★★ ここからが修正箇所 ★★★ ===
    # 結果を「選手の情報(辞書)」のペアのリストに変換します。
    seen = set()
    matching = []
    for i, j in zip(row_ind, col_ind):
        if i not in seen and j not in seen:
            # 以前はIDを返していましたが、選手の情報(辞書)そのものを返します。
            p1_info = current_players[i]
            p2_info = current_players[j]
            matching.append((p1_info, p2_info))
            seen.add(i)
            seen.add(j)

    return matching

In [ ]:
# ==============================================================================
# セクション4: 逆強化学習（IRL）の学習ループ (修正版)
# ==============================================================================
def calculate_feature_expectation(players, matching):
    """
    対戦表全体での、各特徴量の合計値を計算します。
    【修正点】特徴量の数を7個に修正しました。
    """
    player_map = {p['id']: p for p in players}
    # ★★★ 修正箇所 ★★★
    total_features = np.zeros(7) # 特徴量の数を7に

    if not matching: return total_features

    first_pair_item = matching[0][0]
    if isinstance(first_pair_item, dict):
        for p1, p2 in matching:
            total_features += get_feature_vector(p1, p2, set())
    else:
        for id1, id2 in matching:
            p1 = player_map[id1]
            p2 = player_map[id2]
            total_features += get_feature_vector(p1, p2, set())

    return total_features

def train_irl(players, expert_matching, iterations=1000, learning_rate=0.01):
    """
    IRLの学習を実行し、「報酬の重み」を見つけ出します。
    【修正点】特徴量の数を6個に修正しました。
    """
    expert_feature_expectation = calculate_feature_expectation(players, expert_matching)
    print(f"学習目標 (お手本の特徴量合計): \n{expert_feature_expectation}\n")

    # ★★★ 修正箇所 ★★★
    weights = np.zeros(7) # 特徴量の数を6に

    print("--- 学習開始 ---")
    for i in range(iterations):
        ai_matching = solve_matching(players, weights, set())
        ai_feature_expectation = calculate_feature_expectation(players, ai_matching)
        gradient = expert_feature_expectation - ai_feature_expectation
        weights += learning_rate * gradient

        if (i + 1) % 20 == 0:
            print(f"Iteration {i+1}/{iterations}:")
            print(f"  現在の重み: {np.round(weights, 2)}")
            print(f"  目標との誤差: {np.linalg.norm(gradient):.4f}")

    print("--- 学習完了 ---\n")
    return weights

In [ ]:
# ==============================================================================
# セクション5: メイン実行ブロック (修正版)
# ==============================================================================
if __name__ == '__main__':
    # --- 1. 学習の実行 ---
    players_for_training = define_players()
    expert_demo = define_expert_demonstration()

    learned_weights = train_irl(players_for_training,
                                expert_demo,
                                iterations=1000,
                                learning_rate=0.001)

    print(f"最終的に学習された重み: \n{learned_weights}\n")
    print("解説:")
    print(" - 1, 4, 6番目の重みがマイナスに -> 内部対戦, 外部同士, 極端な棋力差を避けるように学習")
    print(" - 2番目の重みがマイナスに -> 棋力差が小さいペアを好むように学習")
    print(" - 3, 5番目の重みがプラスに -> 強化選手, 優先度ボーナスを重視するように学習")

    # --- 2. 学習済みモデルをファイルに保存 ---
    np.save('learned_weights.npy', learned_weights)
    print("\n学習済みの重みを 'learned_weights.npy' として保存しました。\n")


学習目標 (お手本の特徴量合計): 
[ -4. -38.   2.   0.  24.  -2.  -1.]

--- 学習開始 ---
Iteration 20/1000:
  現在の重み: [ 0.01  0.01  0.01  0.    0.06  0.   -0.  ]
  目標との誤差: 23.2379
Iteration 40/1000:
  現在の重み: [ 0.01  0.03  0.01  0.    0.12  0.   -0.01]
  目標との誤差: 43.2001
Iteration 60/1000:
  現在の重み: [ 0.02  0.03  0.01  0.    0.18  0.   -0.02]
  目標との誤差: 45.1221
Iteration 80/1000:
  現在の重み: [ 0.02  0.02  0.02  0.    0.26  0.   -0.02]
  目標との誤差: 23.5000
Iteration 100/1000:
  現在の重み: [ 0.02 -0.02  0.02  0.    0.32  0.   -0.02]
  目標との誤差: 17.1245
Iteration 120/1000:
  現在の重み: [ 0.03  0.05  0.02  0.    0.39  0.   -0.02]
  目標との誤差: 59.1037
Iteration 140/1000:
  現在の重み: [ 0.03 -0.01  0.02  0.    0.45  0.   -0.02]
  目標との誤差: 17.1245
Iteration 160/1000:
  現在の重み: [ 0.03  0.05  0.02  0.    0.52  0.   -0.02]
  目標との誤差: 59.0614
Iteration 180/1000:
  現在の重み: [ 0.03  0.04  0.02  0.    0.59  0.   -0.03]
  目標との誤差: 52.2494
Iteration 200/1000:
  現在の重み: [ 0.04 -0.02  0.03  0.    0.65  0.   -0.03]
  目標との誤差: 17.1245
Iteration 220/1000:
  現在

In [ ]:
# ==============================================================================
# 最終修正版： generate_n_best_matchings 関数
# ==============================================================================
def generate_n_best_matchings(players, weights, past_matchups, num_candidates=5, num_trials=1000):
    """
    質の高い対戦表の候補を複数生成する関数（最終修正版）
    【修正点】
    1. 「再戦禁止」を絶対ルールとして正しく適用。
    2. 「極端な棋力差」を絶対ルールとして、候補から完全に除外。
    """
    current_players = players.copy()

    if len(current_players) % 2 != 0:
        dummy_player = {'id': -1, 'name': '休み', 'university': 'DUMMY', 'rank': 999, 'is_focus': False, 'priority_group': 99}
        current_players.append(dummy_player)

    num_players = len(current_players)
    player_map = {p['id']: p for p in current_players}

    # --- 1. スコア行列を計算 ---
    base_score_matrix = np.zeros((num_players, num_players))
    for i, j in combinations(range(num_players), 2):
        p1 = current_players[i]
        p2 = current_players[j]

        # ダミーとの対戦スコアは0（ニュートラル）
        if p1['id'] == -1 or p2['id'] == -1:
            score = 0
        else:
            # get_feature_vectorからハード制約のチェックをなくし、ソフト制約のみ計算
            features = get_feature_vector(p1, p2, set()) # ここではpast_matchupsは不要
            score = np.dot(weights, features)

        base_score_matrix[i, j] = score
        base_score_matrix[j, i] = score

    # --- 2. ハード制約を適用して、あり得ないペアを候補から除外 ---
    hard_constraints_penalty = 1e9 # 非常に大きなペナルティ（コスト）

    # コスト行列に変換 (スコアが高いほどコストが低い)
    cost_matrix = -base_score_matrix
    np.fill_diagonal(cost_matrix, hard_constraints_penalty) # 自分自身とは対戦しない

    for i, j in combinations(range(num_players), 2):
        p1 = current_players[i]
        p2 = current_players[j]

        # ★★★ 修正箇所1: 再戦ペアに巨大なコストを課す ★★★
        if tuple(sorted((p1['name'], p2['name']))) in past_matchups:
            cost_matrix[i, j] = hard_constraints_penalty
            cost_matrix[j, i] = hard_constraints_penalty

        # ★★★ 修正箇所2: 極端な棋力差のペアに巨大なコストを課す ★★★
        if p1['id'] != -1 and p2['id'] != -1: # ダミーは除く
            if abs(p1['rank'] - p2['rank']) >= 10:
                cost_matrix[i, j] = hard_constraints_penalty
                cost_matrix[j, i] = hard_constraints_penalty

    # --- 3. 何度も試行して、多様な候補を集める ---
    candidate_matchings = {}
    for _ in range(num_trials):
        noise = np.random.normal(0, 1.0, cost_matrix.shape)
        noisy_cost_matrix = cost_matrix + noise

        row_ind, col_ind = linear_sum_assignment(noisy_cost_matrix)

        matching_ids = []
        seen = set()
        for i, j in zip(row_ind, col_ind):
            if i not in seen and j not in seen:
                matching_ids.append(tuple(sorted((current_players[i]['id'], current_players[j]['id']))))
                seen.add(i); seen.add(j)
        candidate_matchings[frozenset(matching_ids)] = 0

    # --- 4. 候補を本来のスコアで再評価 ---
    scored_candidates = []
    for matching_ids in candidate_matchings.keys():
        total_score = 0
        for id1, id2 in matching_ids:
            if id1 == -1 or id2 == -1: continue

            p1 = player_map[id1]; p2 = player_map[id2]
            features = get_feature_vector(p1, p2, set())
            total_score += np.dot(weights, features)

        matching_names = [tuple(sorted((player_map[id1]['name'], player_map[id2]['name']))) for id1, id2 in matching_ids]
        scored_candidates.append({'matching': matching_names, 'score': total_score})

    scored_candidates.sort(key=lambda x: x['score'], reverse=True)
    return scored_candidates[:num_candidates]

In [ ]:
def calculate_plan_A_score(players, weights, past_matchups):
    """通常の最適化プラン（全員真剣勝負）"""
    candidates = generate_n_best_matchings(players, weights, past_matchups, num_candidates=1)
    if not candidates:
        return -1e9, []  # 無効スコア
    return candidates[0]['score'], candidates[0]['matching']


def calculate_plan_B_score(players, weights, past_matchups):
    """監督付きプランを考慮する"""
    # ここはユーザーのルールに合わせて実装
    # 仮に「監督付き4人組を選んでボーナスを加算」とする例
    candidates = generate_n_best_matchings(players, weights, past_matchups, num_candidates=1)
    if not candidates:
        return -1e9, []

    base_score = candidates[0]['score']
    base_matching = candidates[0]['matching']

    # ボーナス例：監督付きがいると+10点
    bonus = 10
    return base_score + bonus, base_matching


In [ ]:
# ==============================================================================
# セクション5: メイン実行ブロック（フェーズ2：人間主導モード対応版）
# ==============================================================================
# 監督付きプランがかなりスコアがよくなっている状態でかなり採択されている状態なので、新川にだけ採用する事をきちんとコードに反映させる必要があるのでは？と考えている。
if __name__ == '__main__':
    # --- 1. 事前準備 ---

    # ▼▼▼ フェーズ2 追加箇所 ▼▼▼
    # 人間が手動で指定する「特別対局」のルールブック
    # ここに、各ラウンドで行わせたい指導対局などを事前に定義しておく
    SPECIAL_ASSIGNMENTS = {
        1: [ # 1局目の特別対局
            {'mode': 'teaching', 'teacher': '真田', 'student': '新川'}
        ],
        2: [ # 2局目の特別対局
            {'mode': 'teaching', 'teacher': '古家', 'student': '新川'}
        ],
        # 3局目以降は、AIに全て任せるので定義しない
    }
    # ▲▲▲ フェーズ2 追加箇所 ▲▲▲

    try:
        LEARNED_WEIGHTS = np.load('learned_weights.npy')
        print("✅ 学習済みの重み 'learned_weights.npy' を読み込みました。")
    except FileNotFoundError:
        print("⚠️ モデルファイルが見つかりません。学習を先に実行してください。")
        exit()

    all_players = define_players()
    player_map_by_name = {p['name']: p for p in all_players}

    NUM_PATTERNS = 3
    all_patterns = []

    # --- 2. 複数パターンの生成ループ ---
    print(f"\nこれから {NUM_PATTERNS} パターンの対戦表を生成します...")

    for pattern_num in range(1,  NUM_PATTERNS + 1):
        print(f"\n========== パターン {pattern_num} の生成 ==========")

        past_matchups = set()
        player_names = [p['name'] for p in all_players]
        final_schedule_df = pd.DataFrame(index=player_names)

        for round_num in range(1, 6):
            print(f"\n--- ラウンド {round_num} の候補生成 ---")

            available_players = []
            for p in all_players:
                if p['name'] == '稲留' and round_num == 1: continue
                if p['name'] in ['堤', '宇治'] and round_num >= 2: continue
                available_players.append(p)

            # ▼▼▼ フェーズ2 追加箇所 ▼▼▼
            # このラウンドで手動指定されたペアを先に確定させる
            fixed_pairs_this_round = []
            players_for_optimization = available_players.copy()

            if round_num in SPECIAL_ASSIGNMENTS:
                print(f"特別ルールを適用します: {SPECIAL_ASSIGNMENTS[round_num]}")
                for assignment in SPECIAL_ASSIGNMENTS[round_num]:
                    teacher_name = assignment['teacher']
                    student_name = assignment['student']

                    # 確定ペアとして保存
                    fixed_pairs_this_round.append(tuple(sorted((teacher_name, student_name))))

                    # 最適化計算の対象から、この2人を除外する
                    players_for_optimization = [p for p in players_for_optimization if p['name'] not in [teacher_name, student_name]]

            # 対戦履歴に、手動で確定させたペアを追加
            for p1_name, p2_name in fixed_pairs_this_round:
                past_matchups.add(tuple(sorted((p1_name, p2_name))))
            # ▲▲▲ フェーズ2 追加箇所 ▲▲▲

            # # b. 残りのメンバーで、上位5件の候補を生成する
            # top_candidates = generate_n_best_matchings(
            #     players_for_optimization, # <-- 変更点: 全員ではなく、残りのメンバー
            #     LEARNED_WEIGHTS,
            #     past_matchups,
            #     num_candidates=5
            # )

            # if not top_candidates:
            #     print("⚠️ 条件を満たす対戦表が見つかりませんでした。")
            #     break

            # # c. 最も良い候補を自動で選択
            # best_optimized_matching = top_candidates[0]['matching']
            # ▼▼▼ 戦略判断フェーズを導入 ▼▼▼
            # プランA: 全員真剣勝負
            score_plan_A, matching_plan_A = calculate_plan_A_score(
                players_for_optimization, LEARNED_WEIGHTS, past_matchups
            )

            # プランB: 「監督付き対局」を考慮した場合
            score_plan_B, matching_plan_B = calculate_plan_B_score(
                players_for_optimization, LEARNED_WEIGHTS, past_matchups
            )

            # どちらを採用するか比較
            if score_plan_B > score_plan_A:
                print("▶️ 戦略判断:『監督付き対局』プランを採用します。")
                best_optimized_matching = matching_plan_B
            else:
                print("▶️ 戦略判断:『全員真剣勝負』プランを採用します。")
                best_optimized_matching = matching_plan_A
            # ▲▲▲ 戦略判断フェーズを導入 ▲▲▲


            # ▼▼▼ フェーズ2 追加箇所 ▼▼▼
            # d. 手動で決めたペアと、AIが最適化したペアを結合する
            best_matching_this_round = fixed_pairs_this_round + best_optimized_matching
            # ▲▲▲ フェーズ2 追加箇所 ▲▲▲

            # e. 対戦履歴を更新
            for p1_name, p2_name in best_optimized_matching:
                past_matchups.add(tuple(sorted((p1_name, p2_name))))

            # f. 結果を記録
            round_opponents = {}
            for p1_name, p2_name in best_matching_this_round:
                round_opponents[p1_name] = p2_name
                round_opponents[p2_name] = p2_name
            for p in available_players:
                if p['name'] not in round_opponents:
                    round_opponents[p['name']] = '(休み)'
            final_schedule_df[f'{round_num}局目'] = final_schedule_df.index.map(round_opponents).fillna('(不参加)')

        all_patterns.append(final_schedule_df)
        print(f"✅ パターン {pattern_num} が完成しました。")

    # --- 3. 最終結果を1つのExcelファイルにまとめて出力 ---
    if all_patterns:
        output_filename = 'final_matchmaking_patterns_phase2.xlsx'
        with pd.ExcelWriter(output_filename) as writer:
            for i, df_pattern in enumerate(all_patterns):
                df_pattern.to_excel(writer, sheet_name=f'パターン_{i+1}')

        print(f"\n\n🎉 全 {len(all_patterns)} パターンの対戦表が完成しました！ 🎉")
        print(f"結果を '{output_filename}' に、それぞれ別のシートとして出力しました。")

✅ 学習済みの重み 'learned_weights.npy' を読み込みました。

これから 3 パターンの対戦表を生成します...

========== パターン 1 の生成 ==========

--- ラウンド 1 の候補生成 ---
特別ルールを適用します: [{'mode': 'teaching', 'teacher': '真田', 'student': '新川'}]
▶️ 戦略判断:『監督付き対局』プランを採用します。

--- ラウンド 2 の候補生成 ---
特別ルールを適用します: [{'mode': 'teaching', 'teacher': '古家', 'student': '新川'}]
▶️ 戦略判断:『監督付き対局』プランを採用します。

--- ラウンド 3 の候補生成 ---
▶️ 戦略判断:『監督付き対局』プランを採用します。

--- ラウンド 4 の候補生成 ---
▶️ 戦略判断:『監督付き対局』プランを採用します。

--- ラウンド 5 の候補生成 ---
▶️ 戦略判断:『監督付き対局』プランを採用します。
✅ パターン 1 が完成しました。

========== パターン 2 の生成 ==========

--- ラウンド 1 の候補生成 ---
特別ルールを適用します: [{'mode': 'teaching', 'teacher': '真田', 'student': '新川'}]
▶️ 戦略判断:『監督付き対局』プランを採用します。

--- ラウンド 2 の候補生成 ---
特別ルールを適用します: [{'mode': 'teaching', 'teacher': '古家', 'student': '新川'}]
▶️ 戦略判断:『監督付き対局』プランを採用します。

--- ラウンド 3 の候補生成 ---
▶️ 戦略判断:『監督付き対局』プランを採用します。

--- ラウンド 4 の候補生成 ---
▶️ 戦略判断:『監督付き対局』プランを採用します。

--- ラウンド 5 の候補生成 ---
▶️ 戦略判断:『監督付き対局』プランを採用します。
✅ パターン 2 が完成しました。

========== パターン 3 の生成 ==========

--- ラウンド 1 の候補生成 ---
特別

In [ ]:
# # ==============================================================================
# # セクション5: メイン実行ブロック（完全版）
# # ==============================================================================

# if __name__ == '__main__':
#     # --- 1. 事前準備 ---

#     # 以前の学習でファイルに保存した「報酬の重み」を読み込む
#     # もし学習をまだ実行していない場合は、このコードの初回実行時に自動生成される
#     try:
#         LEARNED_WEIGHTS = np.load('learned_weights.npy')
#         print("✅ 学習済みの重み 'learned_weights.npy' を読み込みました。")
#     except FileNotFoundError:
#         print("⚠️ モデルファイルが見つかりません。学習を先に実行してください。")
#         # ここでは学習が完了していると仮定して進めます
#         # もしファイルがない状態で実行したい場合は、下の行のコメントを外してください
#         # LEARNED_WEIGHTS = np.array([-964., -49., 8016., -458., 185.])

#     # 合宿に参加する可能性のある全選手を定義
#     all_players = define_players()

#     # 対戦履歴を記憶するための変数。最初は空の状態から始める
#     past_matchups = set()

#     # 全ラウンドの結果を保存するためのDataFrameを準備
#     # 選手名をインデックス（行名）にする
#     player_names = [p['name'] for p in all_players]
#     final_schedule_df = pd.DataFrame(index=player_names)


#     # --- 2. 複数ラウンドの対戦表生成ループ ---

#     # 5回戦までループを実行
#     for round_num in range(1, 6):
#         print(f"\n========== ラウンド {round_num} の候補生成 ==========")

#         # a. この回戦に参加可能な選手を決定する
#         #    ここに途中参加・離脱のルールを記述する
#         available_players = []
#         for p in all_players:
#             # 例：稲留さんは1局目は休み
#             if p['name'] == '稲留' and round_num == 1:
#                 continue
#             # 例：堤さんと宇治さんは2-5局目は休み
#             if p['name'] in ['堤', '宇治'] and round_num >= 2:
#                 continue
#             available_players.append(p)

#         print(f"ラウンド {round_num} の参加者: {[p['name'] for p in available_players]}")

#         # b. 上位5件の候補を生成する
#         top_candidates = generate_n_best_matchings(
#             available_players,
#             LEARNED_WEIGHTS,
#             past_matchups,
#             num_candidates=10, # 表示する候補数
#             num_trials=1000   # 試行回数
#         )

#         # c. 結果を表示する
#         if not top_candidates:
#             print("⚠️ 条件を満たす対戦表が見つかりませんでした。")
#             continue

#         for i, candidate in enumerate(top_candidates):
#             print(f"  --- 候補 {i+1} (スコア: {candidate['score']:.2f}) ---")
#             # print(f"    {candidate['matching']}") # 詳細を見たい場合はコメントを外す

#         # d. 次のラウンドに進むための対戦表を決定する
#         #    ここでは、人間が選ぶ代わりに、スコアが最も高い候補を自動で選択
#         best_matching_this_round = top_candidates[0]['matching']

#         print(f"\n▶️ ラウンド {round_num} の対戦表として候補1を採用します。")

#         # e. 対戦履歴を更新する
#         for p1_name, p2_name in best_matching_this_round:
#             past_matchups.add(tuple(sorted((p1_name, p2_name))))

#         # f. 最終的な対戦表データフレームに結果を記録する
#         round_opponents = {}
#         for p1_name, p2_name in best_matching_this_round:
#             round_opponents[p1_name] = p2_name
#             round_opponents[p2_name] = p1_name

#         # 休みだった選手を探して記録
#         for p in available_players:
#             if p['name'] not in round_opponents:
#                 round_opponents[p['name']] = '(休み)'

#         final_schedule_df[f'{round_num}局目'] = final_schedule_df.index.map(round_opponents).fillna('(不参加)')

#     # --- 3. 最終結果の出力 ---
#     print("\n\n🎉 全5局の対戦表が完成しました！ 🎉")
#     print(final_schedule_df)

#     # 最終的な対戦表をCSVファイルとして出力
#     final_schedule_df.to_csv('final_matchmaking_schedule.csv', encoding='utf-8-sig')
#     print("\n結果を 'final_matchmaking_schedule.csv' に出力しました。")